In [7]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision
import matplotlib.pyplot as plt



folder = ImageFolder('./a5_data_new/train', transform=torchvision.transforms.ToTensor())
loader = DataLoader(folder, batch_size=8, shuffle=True)

# Xexamples, Yexamples = next(iter(loader))

# for i in range(8):
#     plt.subplot(2,4,i+1)  
#     img = Xexamples[i].numpy().transpose(1, 2, 0)    
#     plt.imshow(img, interpolation='none')
#     plt.title('NV' if Yexamples[i] else 'MEL')
#     plt.xticks([])
#     plt.yticks([])
# plt.show()









AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [ ]:
## making the cnn
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm



## first we train the model here is the fuction for training it


# from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
import torchvision

import torch.nn as nn

## lended from lectures evaluates score
def predict_and_evaluate(model, X, Y):

    Xt = torch.tensor(X).type(torch.FloatTensor)
    
    scores = model(Xt).view(-1)
    guesses = (scores > 0).numpy()
    return accuracy_score(Y, guesses)

def train_classifier(model, train_data, val_data, hyperparams):

    # Deals with model updates. Adam is more effective than SGD.
    optimizer = torch.optim.Adam(model.parameters(), lr=hyperparams['lr'])    
    
    # Cross-entropy loss because we have 10 classes.
    # Note that the softmax is "baked into" this loss, so we should not
    # use a softmax at the end.
    loss_func = torch.nn.CrossEntropyLoss()
      
    # Some statistics.
    acc_history = []

    for epoch in range(hyperparams['n_epochs']):

        # Set the model in training mode, enabling dropout if we use that.
        model.train()
        
        loss_sum = 0

        # For each batch
        for Xbatch, Ybatch in tqdm(train_data):

            # Apply the model. We don't know at this point what the model is.
            # The output should be of the shape (batch_size, 10).
            outputs = model(Xbatch)

            # Apply the cross-entropy loss.
            loss = loss_func(outputs, Ybatch)

            # Update the model.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            loss_sum += loss.item()

        # Set the model in evaluation mode. Disables dropout if present.
        model.eval()
        with torch.no_grad():
            # Compute the accuracy on the validation data.
            val_acc = predict_and_evaluate(model, val_data)
                
        mean_loss = loss_sum / len(train_data)

        acc_history.append(val_acc)
        
        print(f'Epoch {epoch+1}: loss = {mean_loss:.4f}, val acc = {val_acc:.4f}')
    
    return acc_history

def train_model(model, Xtrain, Ytrain, Xval, Yval, epochs=10, batch_size=32, learning_rate=0.001):
    # Create DataLoader for training and validation sets
    train_dataset = torch.utils.data.TensorDataset(torch.tensor(Xtrain).float(), torch.tensor(Ytrain).long())
    val_dataset = torch.utils.data.TensorDataset(torch.tensor(Xval).float(), torch.tensor(Yval).long())

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Define hyperparameters
    hyperparams = {
        'n_epochs': epochs,
        'lr': learning_rate
    }

    # Train the model
    acc_history = train_classifier(model, train_loader, val_loader, hyperparams)

    return acc_history
                
## plot the accuracy history
def plot_accuracy_history(acc_history):
    train_acc, val_acc = zip(*acc_history)
    plt.plot(train_acc, label='Train Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

## Import train and test sets from the images. The Y values are the names of the folders
Xtrain = []
Ytrain = []
Xval = []
Yval = []

trainFolder = torchvision.datasets.ImageFolder('./a5_data_new/train', transform=torchvision.transforms.ToTensor())
valFolder = torchvision.datasets.ImageFolder('./a5_data_new/val', transform=torchvision.transforms.ToTensor())

Xtrain, Ytrain = zip(*[(image.numpy().flatten(), label) for image, label in trainFolder])
Xval, Yval = zip(*[(image.numpy().flatten(), label) for image, label in valFolder])

n_input_features = Xtrain[0].shape[0]

# linear_model = nn.Linear(in_features=n_input_features, out_features=1)
# acc_history = train_model(linear_model, Xtrain, Ytrain, Xval, Yval, epochs=10, batch_size=32, learning_rate=0.001)


# Define the PyTorch model
cnn_model = nn.Sequential(
    nn.Conv2d(1, 16, kernel_size=5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(128*128, 50), # adjust input size based on your image dimensions
    nn.ReLU(),
    nn.Linear(50, 10),
)

# Set hyperparameters for grid search
grid_params = {
    'batch_size': [ 32, 64],
    'learning_rate': [0.001, 0.01]
}

# Perform grid search manually
best_score = 0
best_params = {}

for batch_size in grid_params['batch_size']:
    for learning_rate in grid_params['learning_rate']:
        print(f"Training with batch_size={batch_size}, learning_rate={learning_rate}")
        acc_history = train_model(cnn_model, Xtrain, Ytrain, Xval, Yval, epochs=5 , batch_size=batch_size, learning_rate=learning_rate)
        val_acc = acc_history[-1][1]  # Get the validation accuracy of the last epoch
        if val_acc > best_score:
            best_score = val_acc
            best_params = {'batch_size': batch_size, 'learning_rate': learning_rate}

print("Best parameters found: ", best_params)
print("Best validation accuracy: ", best_score)

# Train the model with the best parameters
best_batch_size = best_params['batch_size']
best_learning_rate = best_params['learning_rate']
acc_history = train_model(cnn_model, Xtrain, Ytrain, Xval, Yval, epochs=30, batch_size=best_batch_size, learning_rate=best_learning_rate)

# Plot the accuracy history
plot_accuracy_history(acc_history)


Training with batch_size=32, learning_rate=0.001


RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [32, 49152]